In [315]:
import numpy as np
import constants
import utilities

In [316]:
T = constants.T  # number of minutes in a shift
N = constants.N  # number of resource categories
M = constants.M  # number of shifts
p = constants.p  # patients matrix per resource per shift
x = constants.x  # staff availability matrix
t = constants.t  # time for resource matrix
c = constants.c  # cost of resource matrix

ARRIVAL_TIMES = constants.ARRIVAL_TIMES


In [317]:
def simulation_by_shift(i, j, x, t, o):
    """
    :param i: index of resource [0, 8]
    :param j: index of shift [0, 2]
    :param x: number of staff assigned to resource i in shift j
    :param t: number of minutes it takes for resource i in shift j to serve 1 patient
    :param o: number of patients overflow from previous shifts for resource i in shift j

    :return: number of patients still waiting 
    """
    cur_waiting = o
    staff_availability = dict([x, 0] for x in range(x)) # dict to hold minutes until free for each staff member

    for minute in range(j*T, (j+1)*T):

        if minute in ARRIVAL_TIMES[i]:
            cur_waiting += 1

        x_available = [k for k, v in staff_availability.items() if v == 0] # keys of staff that are free
        if len(x_available) == 0 or minute + t > (j+1)*T:  # if no available staff or not enough time remaining in shift
            pass 
        elif cur_waiting <= len(x_available):          # if enough staff to attend to number of patients waiting
            for k in range(cur_waiting):
                staff_availability[x_available[k]] = t
                served_times[i].append(minute)
            cur_waiting = 0
        else:                                          # if less staff available than waiting patients
            for k in x_available:
                staff_availability[k] = t       
                served_times[i].append(minute)
            cur_waiting -= len(x_available)
            
        # decrement minutes until free at the end of each loop
        for k in staff_availability.keys():
            staff_availability[k] = max(0, staff_availability[k]-1)

    return cur_waiting


def simulation(x, t):
    """Runs simulation per shift for each resource
    :param x: matrix of resources available

    WARNING AHHH: t IS NOT SUPPOSED TO BE PASSED HERE, JUST A TEST FOR NOW
    """
    # N: resource categories
    for i in range(N_test):
        o_1 = simulation_by_shift(i, 0, x[i][0], t[i][0], 0) # shift 1 
        o_2 = simulation_by_shift(i, 1, x[i][1], t[i][1], o_1) # shift 2
        o_3 = simulation_by_shift(i, 2, x[i][2], t[i][2], o_2) # shift 3

        if o_3 != 0:
            return np.inf

    wait_times = [np.array(served_times[i]) - np.array(ARRIVAL_TIMES[i]) for i in range(N)]
    
    return np.sum([np.sum(wait_times[i]) for i in range(N)])


def calculate_objective(x, d, t):
    """ 
    :param x: matrix of number of staff working for each resource in each shift
    
    WARNING AHHH: d IS NOT SUPPOSED TO BE PASSED HERE, WAITING FOR IT TO BE ADDED TO CONSTANTS
    WARNING AHHH: t IS NOT SUPPOSED TO BE PASSED HERE, JUST A TEST FOR NOW

    :return: (value of cost function, list of contraints broken if applicable)
    """

    # calculate resource and wait costs
    resource_cost = sum([c[i][j]*x[i][j] for j in range(M) for i in range(N)])
    wait_cost = simulation(x, t)
    total_cost = resource_cost + d*wait_cost

    if np.isinf(wait_cost):
        return total_cost, True, False
    elif wait_cost > 480:
        return total_cost, False, True

    return total_cost, False, False
    
            

In [318]:
# sample t matrix that works

# wait time: 88 min
test_t_1 = np.array([[ 12,  14,  17],
       [  8,   9,   8],
       [ 50,  50,  20],
       [ 10,  10,  10],
       [  4,   4,   4],
       [  7,   7,   7],
       [ 15,  15,  15],
       [ 30,  30,  30],
       [ 15,  15,  15]])

# wait time: 0 min
test_t_2 = np.array([[ 12,  14,  17],
       [  8,   9,   8],
       [ 30,  30,  20],
       [ 10,  10,  10],
       [  4,   4,   4],
       [  7,   7,   7],
       [ 15,  15,  15],
       [ 30,  30,  30],
       [ 15,  15,  15]])

# wait time: inf
test_t_3 = np.array([[ 12,  14,  17],
       [  8,   9,   8],
       [ 30,  30,  30],
       [ 10,  10,  10],
       [  4,   4,   4],
       [  7,   7,   7],
       [ 15,  15,  15],
       [ 30,  30,  30],
       [ 15,  15,  15]])

# wait time > 480 min
test_t_4 = np.array([[ 12,  14,  17],
       [  8,   9,   8],
       [ 80,  80,  20],
       [ 10,  10,  10],
       [  4,   4,   4],
       [  7,   7,   7],
       [ 15,  15,  15],
       [ 30,  30,  30],
       [ 15,  15,  15]])

# with researched values https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6371290/
test_t_5 = np.array([[ 12,  14,  17],
       [  8,   9,   8],
       [ 21,  20,  14],
       [ 10,  10,  10],
       [  4,   4,   4],
       [  7,   7,   7],
       [ 15,  15,  15],
       [ 30,  30,  30],
       [ 15,  15,  15]])

In [319]:
served_times = [[] for _ in range(N)]
#simulation(x, test_t_1)
#calculate_objective(x, 1, test_t_2)

In [321]:
T = 480  
N = 3  
M = 3  
p = np.array([[4, 4, 4],
             [8, 8, 8],
             [20, 20, 20]])
x = p.copy()
t = np.array([[15, 15, 15],
             [10, 10, 10],
             [5, 5, 5]])
c = np.array([[78, 78, 78],
              [73, 73, 73],
              [39, 39, 39]])

ARRIVAL_TIMES = [[] for _ in range(N)]
for i in range(N):
    for j in range(M):
        padded_time = j*T
        interval = int(np.floor(T/p[i][j]))
        ARRIVAL_TIMES[i].extend(j*T + np.arange(0, interval*p[i][j], interval))

served_times = [[] for _ in range(N)]
simulation(x, t)

0

In [ ]:
#test arrival time 

expected_arrival_times =[0, 24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 264,
                         288, 312, 336, 360, 384, 408, 432, 456]

# Retrieve calculated arrival times for the first 20 patients with 480 minute in a shift
T = 480  # number of minutes in a shift
N = 9  # number of resource categories
M = 3  # number of shifts

p_test =  np.array([[20, 0, 0],
              [0, 0, 0]])
ARRIVAL_TIMES = [[] for _ in range(N)]
for i in range(N):
    for j in range(M):
        padded_time = j * T
        interval = int(np.floor(T / p_test[0][0]))
        ARRIVAL_TIMES[i].extend(j * T + np.arange(0, interval * p_test[0][0], interval))
        
calculated_arrival_times = ARRIVAL_TIMES[0][:20]

# Print the calculated and expected arrival times
print("Calculated arrival times: {}".format(calculated_arrival_times))
print("Expected arrival times: {}".format(expected_arrival_times))


# Compare the calculated and expected arrival times
if np.allclose(calculated_arrival_times, expected_arrival_times):
    print("Arrival times match expectations.")
else:
    print("Arrival times do not match expectations.")


Calculated arrival times: [0, 24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 264, 288, 312, 336, 360, 384, 408, 432, 456]
Expected arrival times: [0, 24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 264, 288, 312, 336, 360, 384, 408, 432, 456]
Arrival times match expectations.
